In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os
from google.colab import drive
from tqdm import tqdm
import collections

# === 1. 설정 ===
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

ROOT_PATH = '/content/drive/MyDrive/Capstone_Project'
DATA_PATH = os.path.join(ROOT_PATH, 'processed_data', 'train_data_30_10.npy')
LABEL_PATH = os.path.join(ROOT_PATH, 'processed_data', 'train_label_30_10.npy')
MODEL_SAVE_BASE = os.path.join(ROOT_PATH, 'experiment_full_metric')

CLASSES = ['Walking', 'Shopping', 'Falldown', 'Threat']
BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 0.01
NUM_RUNS = 5
step_size = 15

# ★★★ 모델 선택 스위치 (여기만 바꾸세요!) ★★★
# 'stgcn' 또는 'lstm'
MODEL_TYPE = 'stgcn'
print(f"▶ 현재 선택된 모델: {MODEL_TYPE.upper()}")


# === 2. 데이터 로드 ===
print("▶ 데이터 로딩...")
X = np.load(DATA_PATH)
Y = np.load(LABEL_PATH)

class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
class_weights = torch.tensor(class_weights, dtype=torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# === 3. 모델 클래스 정의 (두 모델 모두 포함) ===

# [3-1] ST-GCN 관련 클래스
class Graph:
    def __init__(self, num_node=17):
        self.num_node = num_node
        self.edges = [(0,1),(0,2),(1,3),(2,4),(5,7),(7,9),(6,8),(8,10),(5,6),(5,11),(6,12),(11,12),(11,13),(13,15),(12,14),(14,16)]
        self.A = self.get_adjacency_matrix(self.edges, self.num_node)
    def get_adjacency_matrix(self, edges, num_node):
        A = np.zeros((num_node, num_node))
        for i in range(num_node): A[i, i] = 1
        for edge in edges:
            A[edge[0], edge[1]] = 1
            A[edge[1], edge[0]] = 1
        row_sum = np.sum(A, axis=1)
        D_inv_sqrt = np.power(row_sum, -0.5)
        D_inv_sqrt[np.isinf(D_inv_sqrt)] = 0.
        D_mat = np.diag(D_inv_sqrt)
        return torch.tensor(D_mat.dot(A).dot(D_mat), dtype=torch.float32)

class STGCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, A, stride=1, residual=True):
        super().__init__()
        self.A = A
        self.gcn = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.tcn = nn.Sequential(
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=(9,1), padding=(4,0), stride=(stride,1)),
            nn.BatchNorm2d(out_channels), nn.Dropout(0.5, inplace=True)
        )
        if not residual: self.residual_conv = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1): self.residual_conv = lambda x: x
        else: self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride,1))
    def forward(self, x):
        x_spatial = self.gcn(x)
        N, C, T, V = x_spatial.size()
        x_spatial = torch.matmul(x_spatial.view(N, C*T, V), self.A.to(x.device)).view(N, C, T, V)
        return torch.relu(self.tcn(x_spatial) + self.residual_conv(x))

class STGCN_Model(nn.Module):
    def __init__(self, num_classes, in_channels=3, num_frames=30, num_joints=17):
        super().__init__()
        graph = Graph(num_node=num_joints)
        self.A = nn.Parameter(graph.A, requires_grad=False)
        self.data_bn = nn.BatchNorm1d(in_channels * num_joints)

        # Half-STGCN (경량화)
        self.st_gcn_networks = nn.Sequential(
            STGCNBlock(in_channels, 32, self.A),
            STGCNBlock(32, 32, self.A),
            STGCNBlock(32, 32, self.A),
            STGCNBlock(32, 64, self.A, stride=2),
            STGCNBlock(64, 64, self.A),
            STGCNBlock(64, 64, self.A),
            STGCNBlock(64, 128, self.A, stride=2),
            STGCNBlock(128, 128, self.A),
            STGCNBlock(128, 128, self.A)
        )
        self.fcn = nn.Conv2d(128, num_classes, kernel_size=1)

    def forward(self, x):
        N, C, T, V, M = x.size()
        x = x.mean(dim=4) # (N, C, T, V)
        x = x.permute(0, 1, 3, 2).contiguous().view(N, C*V, T)
        x = self.data_bn(x)
        x = x.view(N, C, V, T).permute(0, 1, 3, 2).contiguous()
        x = self.st_gcn_networks(x)
        x = torch.nn.functional.avg_pool2d(x, x.size()[2:])
        return self.fcn(x).view(x.size(0), -1)

# [3-2] LSTM 관련 클래스
class LSTM_Model(nn.Module):
    def __init__(self, num_classes, input_size=34, hidden_size=128, num_layers=2):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x shape: (N, C, T, V, M)
        N, C, T, V, M = x.size()
        x = x.permute(0, 2, 3, 1, 4).contiguous() # (N, T, V, C, M)
        x = x.view(N, T, -1)  # (N, T, Features) Flatten

        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        return self.fc(last_output)


# === 4. 결과 저장소 초기화 ===
metrics_storage = {cls: {'precision': [], 'recall': [], 'f1-score': []} for cls in CLASSES}
metrics_storage['Overall'] = {'accuracy': []}
all_history = {'train_loss': [], 'train_acc': [], 'val_acc': []}
confusion_preds, confusion_labels = [], []

print(f"\n🚀 총 {NUM_RUNS}회 반복 실험 시작 ({MODEL_TYPE.upper()} Model)")

for run in range(1, NUM_RUNS + 1):
    print("-" * 60)
    print(f"🔄 Run {run}/{NUM_RUNS}")

    current_seed = 42 + run
    torch.manual_seed(current_seed)
    np.random.seed(current_seed)

    X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=current_seed, stratify=Y)

    train_data = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
    val_data = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # ★★★ 모델 초기화 분기 로직 (중요!) ★★★
    if MODEL_TYPE.lower() == 'stgcn':
        # ST-GCN은 채널 수(C)가 중요 (보통 3: x,y,score)
        in_channels = X_train.shape[1]
        model = STGCN_Model(num_classes=len(CLASSES), in_channels=in_channels).to(device)
    else:
        # LSTM은 Flatten된 Feature Size가 중요 (C * V)
        input_dim = X_train.shape[1] * X_train.shape[3]
        model = LSTM_Model(num_classes=len(CLASSES), input_size=input_dim).to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size, gamma=0.1)

    best_acc = 0.0
    best_model_path = f"{MODEL_SAVE_BASE}_{MODEL_TYPE}_{run}.pth" # 저장명에 모델타입 추가

    run_train_losses = []
    run_train_accs = []
    run_val_accs = []

    # === 학습 루프 ===
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        train_loop = tqdm(train_loader, desc=f"   Epoch {epoch+1}/{EPOCHS}", leave=False)

        for inputs, labels in train_loop:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            train_loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{100.*train_correct/train_total:.2f}%")

        avg_train_loss = running_loss / len(train_loader)
        avg_train_acc = 100 * train_correct / train_total

        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total
        scheduler.step()

        run_train_losses.append(avg_train_loss)
        run_train_accs.append(avg_train_acc)
        run_val_accs.append(val_acc)

        print(f"   [Run {run}] Ep {epoch+1:02d} | Loss: {avg_train_loss:.4f} | Train Acc: {avg_train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"      └── Best Val Updated! ({best_acc:.2f}%)")

    all_history['train_loss'].append(run_train_losses)
    all_history['train_acc'].append(run_train_accs)
    all_history['val_acc'].append(run_val_accs)

    # === 평가 및 저장 ===
    print(f"\n   ✅ Run {run} Finished. Final Best: {best_acc:.2f}%")
    model.load_state_dict(torch.load(best_model_path))
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    confusion_preds.extend(all_preds)
    confusion_labels.extend(all_labels)

    report = classification_report(all_labels, all_preds, target_names=CLASSES, output_dict=True)
    metrics_storage['Overall']['accuracy'].append(report['accuracy'])
    for cls in CLASSES:
            metrics_storage[cls]['precision'].append(report[cls]['precision'])
            metrics_storage[cls]['recall'].append(report[cls]['recall'])
            metrics_storage[cls]['f1-score'].append(report[cls]['f1-score'])

    print(f"   ✅ Run {run} Summary | Best Val: {best_acc:.2f}% | Fall Recall: {report['Falldown']['recall']:.4f}")

# === 5. 최종 리포트 및 시각화 ===
print("\n" + "=" * 80)
print(f"📊 {MODEL_TYPE.upper()} Model - {NUM_RUNS}회 반복 실험 종합 결과")
print("=" * 80)

acc_mean = np.mean(metrics_storage['Overall']['accuracy']) * 100
acc_std = np.std(metrics_storage['Overall']['accuracy']) * 100
print(f"🏆 Overall Accuracy: {acc_mean:.2f} ± {acc_std:.2f}%")

header = f"{'Class':<12} | {'Metric':<10} | {'Mean':<8} | {'Std':<8}"
print("-" * 45)
print(header)
print("-" * 45)
for cls in CLASSES:
    for metric in ['precision', 'recall', 'f1-score']:
        values = metrics_storage[cls][metric]
        print(f"{cls:<12} | {metric.capitalize():<10} | {np.mean(values):.4f}   | ±{np.std(values):.4f}")
    print("-" * 45)

# === 6. 그래프 그리기 ===
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
mean_loss = np.mean(all_history['train_loss'], axis=0)
std_loss = np.std(all_history['train_loss'], axis=0)
epochs = range(1, EPOCHS + 1)
plt.plot(epochs, mean_loss, 'b-', label='Training Loss')
plt.fill_between(epochs, mean_loss - std_loss, mean_loss + std_loss, color='b', alpha=0.2)
plt.title(f'Training Loss ({MODEL_TYPE.upper()})')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)

plt.subplot(1, 2, 2)
mean_train_acc = np.mean(all_history['train_acc'], axis=0)
mean_val_acc = np.mean(all_history['val_acc'], axis=0)
plt.plot(epochs, mean_train_acc, 'b-', label='Train Acc')
plt.plot(epochs, mean_val_acc, 'r-', label='Val Acc')
plt.title(f'Accuracy ({MODEL_TYPE.upper()})')
plt.xlabel('Epochs'); plt.ylabel('Accuracy (%)'); plt.legend(); plt.grid(True)
plt.tight_layout()
plt.show()

# === 7. 혼돈 행렬 ===
plt.figure(figsize=(8, 6))
cm = confusion_matrix(confusion_labels, confusion_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
plt.title(f'Confusion Matrix ({MODEL_TYPE.upper()})')
plt.ylabel('True Label'); plt.xlabel('Predicted Label')
plt.show()

print("\n✅ 완료! 모델 변경 시 맨 위 'MODEL_TYPE' 변수만 바꾸세요.")

▶ 현재 선택된 모델: STGCN
▶ 데이터 로딩...

🚀 총 5회 반복 실험 시작 (STGCN Model)
------------------------------------------------------------
🔄 Run 1/5


   [Run 1] Ep 01 | Loss: 0.8479 | Train Acc: 58.32% | Val Acc: 54.55%
      └── Best Val Updated! (54.55%)


   [Run 1] Ep 02 | Loss: 0.6277 | Train Acc: 71.21% | Val Acc: 67.28%
      └── Best Val Updated! (67.28%)


   [Run 1] Ep 03 | Loss: 0.5489 | Train Acc: 74.90% | Val Acc: 80.39%
      └── Best Val Updated! (80.39%)

   ✅ Run 1 Finished. Final Best: 80.39%
   ✅ Run 1 Summary | Best Val: 80.39% | Fall Recall: 0.8390
------------------------------------------------------------
🔄 Run 2/5


   [Run 2] Ep 01 | Loss: 0.8526 | Train Acc: 59.75% | Val Acc: 59.92%
      └── Best Val Updated! (59.92%)


...

   [Run 3] Ep 03 | Loss: 0.5352 | Train Acc: 75.75% | Val Acc: 79.63%
      └── Best Val Updated! (79.63%)

   ✅ Run 3 Finished. Final Best: 79.63%
   ✅ Run 3 Summary | Best Val: 79.63% | Fall Recall: 0.7650
------------------------------------------------------------
🔄 Run 4/5


   [Run 4] Ep 01 | Loss: 0.8667 | Train Acc: 57.18% | Val Acc: 70.55%
      └── Best Val Updated! (70.55%)


   [Run 4] Ep 02 | Loss: 0.6163 | Train Acc: 71.00% | Val Acc: 51.03%


   [Run 4] Ep 03 | Loss: 0.5354 | Train Acc: 74.97% | Val Acc: 71.14%
      └── Best Val Updated! (71.14%)

   ✅ Run 4 Finished. Final Best: 71.14%
   ✅ Run 4 Summary | Best Val: 71.14% | Fall Recall: 0.8444
------------------------------------------------------------
🔄 Run 5/5


   Epoch 1/3:  63%|██████▎   | 998/1595 [00:22<00:11, 51.08it/s, acc=54.93%, loss=0.6705]